In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time
from datetime import datetime as dt
from dateutil.parser import parse


# NASA Mars News

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
# URL of page to be scraped
url = 'https://redplanetscience.com/'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')


In [3]:
# Retrieve the parent divs for all articles
results = soup.find_all('div', class_='col-md-8')
# loop over results to get the data
for result in results:
    # scrape the titles and previews 
    news_title = result.find('div', class_='content_title').text
    news_p= result.find('div', class_='article_teaser_body').text
    # printdata
    print(f'''-----------------
    {news_title}
    {news_p} ''')
    

    # Dictionary to be inserted into MongoDB later
    mars = {
        'title': news_title,
        'preview': news_p}

-----------------
    NASA's Treasure Map for Water Ice on Mars
    A new study identifies frozen water just below the Martian surface, where astronauts could easily dig it up. 
-----------------
    Global Storms on Mars Launch Dust Towers Into the Sky
    A Mars Dust Tower Stands Out Dust storms are common on Mars. But every decade or so, something unpredictable happens: a series of runaway storms break out, covering the entire planet in a dusty haze. 
-----------------
    HiRISE Views NASA's InSight and Curiosity on Mars
    New images taken from space offer the clearest orbital glimpse yet of InSight as well as a view of Curiosity rolling along. 
-----------------
    Naming a NASA Mars Rover Can Change Your Life
    Want to name the robotic scientist NASA is sending to Mars in 2020? The student who named Curiosity — the rover currently exploring Mars — will tell you this is an opportunity worth taking. 
-----------------
    NASA's Perseverance Rover Attached to Atlas V Rocket
  

In [4]:
# --- visit the Image website ---
image_url = 'https://spaceimages-mars.com/'
browser.visit(image_url)
time.sleep(2)
html2 = browser.html
soup = bs(html2, 'html.parser')

In [5]:
results = soup.find_all('div', class_='header')
# loop over results to get the data
for result in results:
    image_address = image_url + result.find('img', class_='headerimage fade-in')['src']
    print(image_address)

https://spaceimages-mars.com/image/featured/mars1.jpg


# Mars facts

In [6]:
url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(url)
table= tables[0]
table1=table.rename(columns=table.iloc[0]).drop(table.index[0]) 
html_table1 = table1.to_html().replace('\n', '')
html_table1

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Mars - Earth Comparison</th>      <th>Mars</th>      <th>Earth</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>Diameter:</td>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>4</th>      <td>Distance from Sun:</td>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>5</th>      <td>Length of Year:</td>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>6</th>      <td>Temperature:</td>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [7]:
Table2= tables[1]
table2=Table2.rename(columns=Table2.iloc[0]).drop(Table2.index[0]) 
html_table2 = table2.to_html().replace('\n', '')
html_table2

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Equatorial Diameter:</th>      <th>6,792 km</th>    </tr>  </thead>  <tbody>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

# Mars hemispheres

In [8]:
# ---Mars Hemisphere website ---
Marsurl = 'https://marshemispheres.com/'
browser.visit(Marsurl)
time.sleep(2)
html = browser.html
soup = bs(html, 'html.parser')

In [9]:
# Mars hemispheres products data
mars_hemispheres = soup.find_all('div', class_='item')

hemisphere_image_urls = []

# Iterate through each hemisphere data
for hemisphere in mars_hemispheres:
    title=hemisphere.find('h3').text.strip()
    image_url = [Marsurl+hemisphere.find('a')['href']]
    for image in image_url:
        browser.visit(image)
        time.sleep(2)
        html = browser.html
        soup = bs(html, 'html.parser')
        lastlink =Marsurl+soup.find('img', class_='wide-image')['src']

    # Create Dictionary to store title and url info
    image_dict = {
        'title': title,
        'img_url': lastlink}
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]
